# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных**
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Откроем файл с данными и изучим общую информацию

In [1]:
import pandas as pd 
from pymystem3 import Mystem 
from collections import Counter 

Импортируем и изучим датасет.

In [2]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv') 

In [3]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


В колонке days_employed (общий трудовой стаж в днях) наблюдаются артефакты - отрицательные значения.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Определим уникальные значения в каждом столбце данных, кроме столбцов "days_employed" и "total_income" из-за нерациональности просмотра этих данных.

In [5]:
for i in data.columns.drop(['days_employed', 'total_income']):
    print(f'Уникальные значения в столбце {str(i)}: {data[i].unique()}')

Уникальные значения в столбце children: [ 1  0  3  2 -1  4 20  5]
Уникальные значения в столбце dob_years: [42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
Уникальные значения в столбце education: ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
Уникальные значения в столбце education_id: [0 1 2 3 4]
Уникальные значения в столбце family_status: ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
Уникальные значения в столбце family_status_id: [0 1 2 3 4]
Уникальные значения в столбце gender: ['F' 'M' 'XNA']
Уникальные значения в столбце income_type: ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
У

Проверим артефакты в столбце 'количество детей в семье'.

In [6]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Проверим артефакты в столбце 'пол клиента'.

In [7]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

### Вывод

**1.1.** В таблице имеются пропуски (число ненулевых значений в некоторых столбцах отличается от количества строк).  
**1.2.** В столбце "days_employed" имеются артефакты - отрицательные значения. Вероятная причина возникновения преобразование данных из одного типа в другой.  
**1.3.** В столбце "education" данные записаны разным регистром. Вероятная причина возникновения человеческий фактор при заполнении.  
**1.4.** В столбце "purpose" одна и та же по смыслу цель записана разными способами (например "покупка своего жилья" и "покупка жилья"). Вероятная причина возникновения человеческий фактор при заполнении.  
**1.5.** В столбце "children" имеются артефакты - отрицательные значения. Вероятная причина возникновения человеческий фактор при заполнении.   
**1.6.** В столбце "gender" имеется неизвестное значение xna. Вероятная причина возникновения человеческий фактор при заполнении. 
 

## Предобработка данных

### Обработка пропусков

Так как перед нами не стоит задача определить кореляцию кредитоспособности от общего трудового стажа, чтобы не исключать строки с данными целиком, обработаем пропуски данных в столбце "days_employed" ad-hoc методом - заполнением пропусков нулями.

In [8]:
data['days_employed'] = data['days_employed'].fillna(0)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Стобец "total_income" представляет для нас интерес в связи с поставленными задачами исследования. Для решения способа обработки пропусков найдем максимальное и минимальное значение в столбце.

In [9]:
print("Максимальное значение ежемесячного дохода: {:.0f}".format(data['total_income'].max())) 
print("Минимальное значение ежемесячного дохода: {:.0f}".format(data['total_income'].min()))  
print("Отношение максимального ежемесячного дохода к минимальному: {:.0f}".format(
    data['total_income'].max()/ data['total_income'].min())) 

Максимальное значение ежемесячного дохода: 2265604
Минимальное значение ежемесячного дохода: 20667
Отношение максимального ежемесячного дохода к минимальному: 110


В связи с огромным разрывом (более чем в 100 раз) между максимальным и минимальным значением ежемесячного дохода используем другую разновидность ad-hoc метода - заполнение медианной, так как заполнение средним значением или нулями приведет к более существенным искажениям выборки. Заменим пропуски в total_incom на медианное значение группируя по income_type. 

In [10]:
data.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [11]:
data['total_income'] = data[['total_income']].fillna(data.groupby("income_type").transform("median")) 

Проверим пропуски.

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Заменим значение (-1) в столбце с детьми на 1 методом .replace(). Данная замена обусловленна предположением о появлении ошибочного знака "-" при вводе значения 1.

In [13]:
data['children'] = data['children'].replace(-1, 1)

Проверим уникальные значение в колонке 'children'.

In [14]:
data['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

**Вывод**

Все пропущенные значения типа "NaN" в стоблцах 'days_employed' и 'total_income' заполнены нулями и медианой по группам income_type соответственно. Значение (-1) в стоблце 'children' заменено на (1).  
Так как идентифицировать пол клиента в строках с данными 'xna' по имеющимся данным не представляется возможным и перед нами не стоит исcледовательских задач касающихся пола клиента оставим данные без изменения.

### Замена типа данных

Избавимся от артефактов с отрицательными значениями в столбце 'days_employed' вернув абсолютную величину функцией abs.  
Альтернативным способом является умножение отрицательных значений на (-1). Для поиска отрицальных значений в столбце воспользуемся атрибутом .loc. Оставим только отрицательные значения в столбце и присвоим им то же значение умноженное на (-1). Однако этот способ длиннее и поэтому менее предпочтителен. 

In [15]:
data['days_employed'] = abs(data['days_employed']) 
#data.loc[data['days_employed'] < 0,'days_employed'] = data['days_employed'] * (-1) # альтернативный способ

Для наглядности и удобства заменим типы данных в столбцах 'days_employed' и 'total_income' с вещественного на целочисленый методом .astype('int').

In [16]:
data['days_employed'] = data['days_employed'].astype('int') 
data['total_income'] = data['total_income'].astype('int') 

In [17]:
data.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

Проверим результат на 5 первых значениях столбцов 'days_employed' и 'total_income'.

In [18]:
data[['days_employed', 'total_income']].head()

,days_employed,total_income
0,8437,253875
1,4024,112080
2,5623,145885
3,4124,267628
4,340266,158616


Проверим наличие отрицательных значений в столбце 'days_employed'

In [19]:
if data[data['days_employed'] < 0]['days_employed'].count() > 0: 
    print('В столбце "days_employed" присутствуют отрицательные значения')
else:
    print('Отрицательных значений в столбце days_employed нет.')

Отрицательных значений в столбце days_employed нет.


**Вывод**

Для удобства и наглядности анализа данных значения в столбцах 'days_employed' и 'total_income' заменены на целочисленные значения методом .astype('int') 

### Обработка дубликатов

В стобцах 'family_status' и 'education' имеются записи в разных регистрах. Переведем все данные в этих стобцах к одному виду записи - нижнему регистру.

In [20]:
data['family_status'] = data['family_status'].str.lower() 
data['education'] = data['education'].str.lower()  
print('Уникальные значения в столбце "семейное положение": ', data['family_status'].unique())
print('Уникальные значения в столбце "уровень образования клиента": ', data['education'].unique())

Уникальные значения в столбце "семейное положение":  ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']
Уникальные значения в столбце "уровень образования клиента":  ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


Обработаем дубликаты в таблице.

Выведем число дублирующихся строк.

In [21]:
data.duplicated().sum()

71

Удалим дубликаты методом .drop_duplicates() в связке с .reset_index() чтобы не создавать столбец со старыми значениями индексов.

In [22]:
data = data.drop_duplicates().reset_index(drop=True)

**Вывод**

Дублирующиеся значения приведены к общему виду написания в нижнем регистре. Выявлена и удалена 71 дублирующаяся строка. Возможные причины возникновения дубликатов в человеческий фактор, ошибки при конвертировании датафрейма.  

### Лемматизация

Объявляем фунцию для лемматизации строк.

In [23]:
m = Mystem()
def lemmatization(row):
    lemm = m.lemmatize(row)
    return lemm

Применяемя функцию лемматизации для колоники purpose (цель кредита). Сохраняем леммы целей кредита в колонку purpose_lemm.

In [24]:
data['purpose_lemm'] = data['purpose'].apply(lemmatization) 

Выведем получившиеся леммы и число их упоминаний.

In [25]:
print(Counter(data['purpose_lemm'].sum())) 

Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


**Вывод**

С помощью лемматизации столбца "purpose" выделены леммы в значениях столбца с целями получения кредита. Лемматизирование значения сохранены в колонке 'purpose_lemm'.  
Основные категории целей получение кредита: недвижимость, жилье, автомобиль, образование, свадьба, строительство, ремонт. 

### Категоризация данных

**Этап 1.** В соответствии с задачами исследования произведем категоризацию данных по столбцу "children". Выделим две группы клиентов: "дети есть" и "детей нет".

Объявляем функцию для определения категоризации клинетов по признаку наличия детей.

In [26]:
def children_category(row):
    if row == 0:
        return 'детей нет'
    return 'дети есть' 

Применяем функцию к столбцу "children" и сохраняем в новый столбец "children_status"

In [27]:
data['children_status'] = data['children'].apply(children_category)

Проверяем функцию категоризации клиентов по группам "дети есть" и "детей нет"

In [28]:
data['children_status'].value_counts()

детей нет    14091
дети есть     7363
Name: children_status, dtype: int64

**Этап 2.** Произведем категоризацию данных по столбцу "family_status_id". Для этого составим словарь с идентификаторами  'family_status_id' и 'family_status'.

In [29]:
print('Словарь для определения семеного положения')
family_dictionary = data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)
family_dictionary

Словарь для определения семеного положения


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


Считаем количество клиентов в каждой категории по семеному положению методом .value_counts()

In [30]:
data['family_status_id'].value_counts(sort=False)

0    12339
1     4151
2      959
3     1195
4     2810
Name: family_status_id, dtype: int64

**Этап 3.** Произведем категоризацию данных по столбцу "total_income". Распределим клиентов по доходу следующим образом "низкий уровень дохода" "средний уровень дохода" и "высокий уровень дохода".

Категоризацию клиетов по уровню доходу выполним применение метода .quantile() к значениям столбца "total_income". Так как категоризация будет осуществляться на три группы, то в качестве аргументов методода .quantile() применим 0.33 и 0.66, остальных отнесем в категорию с высоким доходом. Таким образом мы поделим клинетов на три примерно равные группы.

In [31]:
low_income = (data['total_income']).quantile(.33) 
mean_income = (data['total_income']).quantile(.66) 

Объявляем функцию для определения категорий дохода.

In [32]:
def income(total_income):
    if total_income <= low_income:
        return "низкий уровень дохода"
    if total_income <= mean_income:
        return "средний уровень дохода"
    return "высокий уровень дохода"

In [33]:
print(f"Если среднемесячный доход не более {low_income:.0f}, то клиент определяется в категорию '{income(low_income)}'")
print(f"Если среднемесячный доход не более {mean_income:.0f}, то клиент определяется в категорию '{income(mean_income)}'")
print(f"Если среднемесячный доход более {mean_income:.0f}, то клиент определяется в категорию '{income(mean_income + 0.1)}'")

Если среднемесячный доход не более 118514, то клиент определяется в категорию 'низкий уровень дохода'
Если среднемесячный доход не более 172357, то клиент определяется в категорию 'средний уровень дохода'
Если среднемесячный доход более 172357, то клиент определяется в категорию 'высокий уровень дохода'


Применяем к столбцу 'total_income' функцию для определения категорий дохода и сохраняем в столбец 'income_group'.

In [34]:
data['income_group'] = data['total_income'].apply(income)

Проверяем значения нового столбца для определения категорий дохода.

In [35]:
data['income_group'].value_counts()

средний уровень дохода    7341
низкий уровень дохода     7096
высокий уровень дохода    7017
Name: income_group, dtype: int64

Для категоризации данных из столбца 'purpose' (цель получения кредита), выбираем ключевые леммы и собираем из них список. После этого создаем функцию категоризации на основе проверки наличия ключевых лемм в стороках.

In [36]:
lemm_list = ['ремонт', 'строительство', 'свадьба', 'жилье', 'образование', 'автомобиль', 'недвижимость'] 

def purpose_categorize(row):
    for word in lemm_list:
        if word in row:
            return word

Применяем функцию категоризации для целей получения кредита по лемматизированным строкам и присвоим ламматизированные цели в столбец 'purpose_category'.

In [37]:
data['purpose_category'] = data['purpose_lemm'].apply(purpose_categorize)

Проверяем категоризацию клиентов по целям получения кредита недвижимость.

In [38]:
data['purpose_category'].value_counts()

недвижимость     4473
автомобиль       4306
образование      4013
жилье            3853
свадьба          2324
строительство    1878
ремонт            607
Name: purpose_category, dtype: int64

**Вывод**

Данные, необходимые для решения поставленных задач категорированы. Созданы столбцы для категоризации по наличию детей, среднемесячному доходу, целей кредита. Составлен словарь для определения семейного статуса клинета.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [39]:
data_pivot_children = data.pivot_table(index=['children_status'], values='debt', aggfunc='mean')
data_pivot_children.style.format({'debt': '{:.2%}'})

,debt
children_status,
детей нет,7.54%
дети есть,9.21%


**Вывод**

9,21% клиентов в категории "дети есть" имеют задолженность по возврату кредита.  
7,54% клиентов в категории "детей нет" имеют задолженность по возврату кредита.  
Наличие детей увеличивает вероятность невозврата кредита в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [40]:
data_pivot_family = data.pivot_table(index=['family_status'], values='debt', aggfunc='mean')
data_pivot_family.sort_values(by = 'debt', ascending=False).style.format({'debt': '{:.2%}'})

,debt
family_status,
не женат / не замужем,9.75%
гражданский брак,9.35%
женат / замужем,7.55%
в разводе,7.11%
вдовец / вдова,6.57%


**Вывод**

Наименьшая доля невозрата кредита в срок наблюдается в категории "вдовец / вдова" - 6,57%.  
Наибольшая доля невозрата кредита в срок наблюдается в категорий "не женат / не замужем" и "гражданский брак"  - 9,75% и 9,35% соответственно.  
Семеное положение существенно влияет на возврат кредита в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [41]:
data_pivot_income = data.pivot_table(index=['income_group'], values='debt', aggfunc='mean')
data_pivot_income.sort_values(by = 'debt', ascending=False).style.format({'debt': '{:.2%}'})

,debt
income_group,
средний уровень дохода,8.64%
низкий уровень дохода,8.16%
высокий уровень дохода,7.52%


**Вывод**

Наименьшая доля невозрата кредита в срок наблюдается в категории "высокий уровень дохода" - 7,52%.  
Наибольшая доля невозрата кредита в срок наблюдается в категории "средний уровень дохода" -  8,64%.  
Уровень дохода менее всего влияет на возврат кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [42]:
data_pivot_purpose = data.pivot_table(index=['purpose_category'], values='debt', aggfunc='mean')
data_pivot_purpose.sort_values(by = 'debt', ascending=False).style.format({'debt': '{:.2%}'})

,debt
purpose_category,
автомобиль,9.36%
образование,9.22%
свадьба,8.00%
строительство,7.67%
недвижимость,7.38%
жилье,7.09%
ремонт,5.77%


**Вывод**

Наименьшая доля невозрата кредита в срок наблюдается в категории "ремонт" - 5,77%.
Наибольшая доля невозрата кредита в срок наблюдается в категории "автомобиль" - 9,36% и образование 9,22%.  
Цели кредита существенно влияют на его возврат в срок.

## Шаг 4. Общий вывод

Проведенное исследование надежности заемщиков позволило выявить степень влияния семеного положения и количества детей на факт погашения кредита в срок. Анализ данных может позволить составить рейтинг заемщиков исходя из их принадлежности к той или иной категори. Например клиент с семеным положением вдовец или вдова без детей с высоким доходом и желающий взять кредит на жилье будет иметь наибольшие шансы вернуть кредит в срок.